In [1]:
# Import Dependencies
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import pandas as pd

In [2]:
# Import Csv

file_to_load = 'Resources/Mental_Health_Programs.csv'

# Convert to DF
programs_df = pd.read_csv(file_to_load)
programs_df

,X,Y,OBJECTID,source,ext_id,cat1,cat2,cat3,org_name,Name,...,zip,link,use_type,latitude,longitude,date_updated,email,dis_status,POINT_X,POINT_Y
0,-118.136071,34.695266,1,LA County Department of Mental Health,7673.0,Health and Mental Health,Mental Health Programs,NaN,NaN,Residential Program,...,93534.0,http://egis3.lacounty.gov/lms/?p=215,publish,34.695266,-118.136071,2010/11/01 19:50:56+00,NaN,NaN,6.520770e+06,2.075430e+06
1,-118.544522,34.379410,5,LA County Department of Mental Health,7413.0,Health and Mental Health,Mental Health Programs,NaN,NaN,Olive Vista,...,91321.0,http://egis3.lacounty.gov/lms/?p=541,publish,34.379410,-118.544522,2010/11/01 19:50:56+00,NaN,NaN,6.397374e+06,1.960900e+06
2,-118.486827,34.199690,14,211,NaN,Health and Mental Health,Mental Health Programs,NaN,NaN,Valley Trauma Counseling Center - Sophia Ave V...,...,91406.0,http://egis3.lacounty.gov/lms/?p=563,publish,34.199690,-118.486827,2013/06/01 11:50:56+00,NaN,NaN,6.414465e+06,1.895408e+06
3,-118.257443,34.128058,26,211,NaN,Health and Mental Health,Mental Health Programs,NaN,NaN,Glendale Memorial Hospital And Health Center,...,91204.0,http://egis3.lacounty.gov/lms/?p=570,publish,34.128058,-118.257443,2016/01/22 16:10:02+00,NaN,NaN,6.483758e+06,1.869083e+06
4,-118.265746,34.037364,55,211,NaN,Health and Mental Health,Mental Health Programs,NaN,Catholic Healthcare West,California Hospital Medical Center,...,90015.0,http://egis3.lacounty.gov/lms/?p=574,publish,34.037364,-118.265746,2016/01/21 17:29:05+00,NaN,NaN,6.481158e+06,1.836084e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,-118.203108,33.929844,65740,NaN,NaN,Health and Mental Health,Mental Health Programs,NaN,St. Francis Medical Center 3630,St. Francis Medical Center,...,90262.0,http://egis3.lacounty.gov/lms/?p=72050,publish,33.929844,-118.203108,2013/08/21 11:23:29+00,NaN,NaN,6.500056e+06,1.796911e+06
285,-118.309063,34.015125,65757,NaN,NaN,Health and Mental Health,Mental Health Programs,NaN,Pacific Clinics - Portals,Pacific Clinics/Portals,...,90052.0,http://egis3.lacounty.gov/lms/?p=72076,publish,34.015125,-118.309062,2013/09/11 08:46:40+00,NaN,NaN,6.468010e+06,1.828029e+06
286,-118.254735,33.945678,65763,NaN,NaN,Health and Mental Health,Mental Health Programs,NaN,NAMI (National Alliance for the Mentally Ill) ...,NAMI (National Alliance for the Mentally Ill) ...,...,90047.0,http://egis3.lacounty.gov/lms/?p=72117,publish,33.945678,-118.254735,2013/09/11 09:48:03+00,NaN,NaN,6.484410e+06,1.802709e+06
287,-118.207198,33.904867,65810,NaN,NaN,Health and Mental Health,Mental Health Programs,NaN,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,...,90221.0,http://egis3.lacounty.gov/lms/?p=72322,publish,33.904867,-118.207198,2013/08/14 08:21:34+00,NaN,NaN,6.498797e+06,1.787824e+06


In [3]:
# Examining the columns
programs_df.columns

Index(['X', 'Y', 'OBJECTID', 'source', 'ext_id', 'cat1', 'cat2', 'cat3',
       'org_name', 'Name', 'addrln1', 'addrln2', 'city', 'state', 'hours',
       'phones', 'url', 'info1', 'info2', 'post_id', 'description', 'zip',
       'link', 'use_type', 'latitude', 'longitude', 'date_updated', 'email',
       'dis_status', 'POINT_X', 'POINT_Y'],
      dtype='object')

In [4]:
# Drop Null columns
programs_df = programs_df.drop(['email', 'source', 'link', 'use_type', 'cat3', 'description', 'ext_id', 'addrln1', 'addrln2', 'info1', 'info2', 'hours', 'phones'],axis = 1)
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,url,post_id,zip,latitude,longitude,date_updated,dis_status,POINT_X,POINT_Y
0,-118.136071,34.695266,1,Health and Mental Health,Mental Health Programs,NaN,Residential Program,Lancaster,CA,NaN,215,93534.0,34.695266,-118.136071,2010/11/01 19:50:56+00,NaN,6.520770e+06,2.075430e+06
1,-118.544522,34.379410,5,Health and Mental Health,Mental Health Programs,NaN,Olive Vista,Newhall,CA,NaN,541,91321.0,34.379410,-118.544522,2010/11/01 19:50:56+00,NaN,6.397374e+06,1.960900e+06
2,-118.486827,34.199690,14,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center - Sophia Ave V...,Van Nuys,CA,www.csun.edu/vtc,563,91406.0,34.199690,-118.486827,2013/06/01 11:50:56+00,NaN,6.414465e+06,1.895408e+06
3,-118.257443,34.128058,26,Health and Mental Health,Mental Health Programs,NaN,Glendale Memorial Hospital And Health Center,Glendale,CA,http://www.glendalememorial.com/,570,91204.0,34.128058,-118.257443,2016/01/22 16:10:02+00,NaN,6.483758e+06,1.869083e+06
4,-118.265746,34.037364,55,Health and Mental Health,Mental Health Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,http://www.chmcla.org,574,90015.0,34.037364,-118.265746,2016/01/21 17:29:05+00,NaN,6.481158e+06,1.836084e+06
5,-118.186744,33.808107,82,Health and Mental Health,Mental Health Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,http://memorialcare.org/locations/search/adult...,577,90806.0,33.808107,-118.186744,2016/01/25 13:37:43+00,NaN,6.504938e+06,1.752599e+06
6,-118.588723,34.437405,96,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center - Santa Clarit...,Valencia,CA,www.csun.edu/vtc,581,91355.0,34.437405,-118.588723,2013/06/01 11:50:56+00,NaN,6.384161e+06,1.982080e+06
7,-118.307462,33.738067,108,Health and Mental Health,Mental Health Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,california.providence.org/san-pedro/Pages/defa...,591,90732.0,33.738067,-118.307462,2017/10/31 16:49:05+00,NaN,6.468187e+06,1.727199e+06
8,-118.535918,34.228586,143,Health and Mental Health,Mental Health Programs,NaN,Valley Trauma Counseling Center,Northridge,CA,www.csun.edu/vtc,614,91324.0,34.228586,-118.535918,2013/06/01 11:50:56+00,NaN,6.399677e+06,1.905999e+06
9,-118.310283,33.928430,144,Health and Mental Health,Mental Health Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,www.shieldsforfamilies.org,619,90047.0,33.928430,-118.310283,2016/05/16 08:38:19+00,NaN,6.467543e+06,1.796480e+06


In [5]:
programs_df.count()

X               289
Y               289
OBJECTID        289
cat1            289
cat2            289
org_name         69
Name            289
city            288
state           288
url             214
post_id         289
zip             288
latitude        289
longitude       289
date_updated    289
dis_status        0
POINT_X         289
POINT_Y         289
dtype: int64

In [6]:
programs_df = programs_df.drop(['org_name', 'dis_status', 'url'], axis=1)

In [7]:
programs_df.count()

X               289
Y               289
OBJECTID        289
cat1            289
cat2            289
Name            289
city            288
state           288
post_id         289
zip             288
latitude        289
longitude       289
date_updated    289
POINT_X         289
POINT_Y         289
dtype: int64

In [8]:
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.136071,34.695266,1,Health and Mental Health,Mental Health Programs,Residential Program,Lancaster,CA,215,93534.0,34.695266,-118.136071,2010/11/01 19:50:56+00,6.520770e+06,2.075430e+06
1,-118.544522,34.379410,5,Health and Mental Health,Mental Health Programs,Olive Vista,Newhall,CA,541,91321.0,34.379410,-118.544522,2010/11/01 19:50:56+00,6.397374e+06,1.960900e+06
2,-118.486827,34.199690,14,Health and Mental Health,Mental Health Programs,Valley Trauma Counseling Center - Sophia Ave V...,Van Nuys,CA,563,91406.0,34.199690,-118.486827,2013/06/01 11:50:56+00,6.414465e+06,1.895408e+06
3,-118.257443,34.128058,26,Health and Mental Health,Mental Health Programs,Glendale Memorial Hospital And Health Center,Glendale,CA,570,91204.0,34.128058,-118.257443,2016/01/22 16:10:02+00,6.483758e+06,1.869083e+06
4,-118.265746,34.037364,55,Health and Mental Health,Mental Health Programs,California Hospital Medical Center,Los Angeles,CA,574,90015.0,34.037364,-118.265746,2016/01/21 17:29:05+00,6.481158e+06,1.836084e+06
5,-118.186744,33.808107,82,Health and Mental Health,Mental Health Programs,Long Beach Memorial Medical Center,Long Beach,CA,577,90806.0,33.808107,-118.186744,2016/01/25 13:37:43+00,6.504938e+06,1.752599e+06
6,-118.588723,34.437405,96,Health and Mental Health,Mental Health Programs,Valley Trauma Counseling Center - Santa Clarit...,Valencia,CA,581,91355.0,34.437405,-118.588723,2013/06/01 11:50:56+00,6.384161e+06,1.982080e+06
7,-118.307462,33.738067,108,Health and Mental Health,Mental Health Programs,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,591,90732.0,33.738067,-118.307462,2017/10/31 16:49:05+00,6.468187e+06,1.727199e+06
8,-118.535918,34.228586,143,Health and Mental Health,Mental Health Programs,Valley Trauma Counseling Center,Northridge,CA,614,91324.0,34.228586,-118.535918,2013/06/01 11:50:56+00,6.399677e+06,1.905999e+06
9,-118.310283,33.928430,144,Health and Mental Health,Mental Health Programs,Shields For Families Inc. - Tamar Village,Los Angeles,CA,619,90047.0,33.928430,-118.310283,2016/05/16 08:38:19+00,6.467543e+06,1.796480e+06


In [10]:
print(programs_df['city'].unique())

['Lancaster' 'Newhall' 'Van Nuys' 'Glendale' 'Los Angeles' 'Long Beach'
 'Valencia' 'San Pedro' 'Northridge' 'Rosemead' 'Hawthorne' 'Pasadena'
 'Irwindale' 'Covina' 'Lakewood' 'Walnut' 'Lynwood' 'Downey'
 'South Pasadena' 'Commerce' 'Monrovia' 'Sylmar' 'Torrance'
 'Woodland Hills' 'Palmdale' 'Sherman Oaks' 'Canoga Park' 'Venice'
 'Hawaiian Gardens' 'Inglewood' 'City Of Commerce' 'Reseda' 'Chatsworth'
 'El Monte' 'Pomona' 'Redondo Beach' 'Santa Monica' 'San Gabriel'
 'Baldwin Park' 'Encino' 'Pacoima' 'Culver City' 'Arcadia' 'Gardena'
 'Burbank' nan 'Los Angeles ' 'Compton' 'Signal Hill ' 'Santa Fe Springs'
 'Glendora' 'West Covina' 'La Verne' 'Pico Rivera' 'Huntington Park'
 'Cerritos' 'Irvine' 'North Hills' 'Granada Hills' 'North Hollywood'
 'Santa Clarita' 'Cypress' 'San Fernando' 'Whittier' 'City of Commerce'
 'La Puente' 'Mission Hills' 'La Habra' 'Tarzana' 'Bellflower' 'Carson'
 'Montebello' 'Los Angelels']
